In [23]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras.layers import Input, Flatten, Conv2D, Conv1D, MaxPooling2D, MaxPooling1D
from cancer_cnn_generator import DataGenerator
import pickle
from  sklearn.model_selection import train_test_split
import h5py

In [24]:
num_epochs = 6
steps_per_epoch = 5000

input_shape = (128,128,1)

IDs = pickle.load(open("cancer_IDs", "rb"))
train_IDs, test_IDs = train_test_split(IDs, test_size=0.2)
train_IDs, val_IDs = train_test_split(train_IDs, test_size=0.15)


params = {'dim': input_shape, 'batch_size': 30 , 'shuffle': True}

label_key = 'patient_tss'

In [25]:
train_generator = DataGenerator(train_IDs, label_key, **params)
val_generator = DataGenerator(val_IDs, label_key, **params)
test_generator = DataGenerator(test_IDs, label_key, **params)

num_classes = train_generator.get_num_classes()

In [45]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(8, 8), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (4, 4), activation='relu'))
#model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
#model.add(Reshape((-1,1)))
#model.add(Conv1D(64, 3, activation='relu'))
#model.add(MaxPooling1D(3))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
#model.add(Reshape((-1,1)))
#model.add(Conv1D(64, 3, activation='relu'))
#model.add(MaxPooling1D(3))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [46]:
print('==>Compiling Model.')
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


==>Compiling Model.


In [47]:
print('==>Fitting Model.')
model.fit_generator(generator=train_generator, 
    steps_per_epoch=steps_per_epoch, epochs=num_epochs, 
    validation_data=val_generator, validation_steps=500,
    use_multiprocessing=True, workers=4)


==>Fitting Model.
Epoch 1/6
5000/5000 [==============================] - 3572s 714ms/step - loss: 0.0727 - acc: 0.6266 - val_loss: 0.0732 - val_acc: 0.6215
Epoch 2/6
5000/5000 [==============================] - 3572s 714ms/step - loss: 0.0724 - acc: 0.6280 - val_loss: 0.0733 - val_acc: 0.6215
Epoch 3/6
5000/5000 [==============================] - 3580s 716ms/step - loss: 0.0726 - acc: 0.6263 - val_loss: 0.0732 - val_acc: 0.6215
Epoch 4/6
5000/5000 [==============================] - 3573s 715ms/step - loss: 0.0723 - acc: 0.6282 - val_loss: 0.0732 - val_acc: 0.6215
Epoch 5/6
5000/5000 [==============================] - 3573s 715ms/step - loss: 0.0723 - acc: 0.6285 - val_loss: 0.0733 - val_acc: 0.6215
Epoch 6/6
5000/5000 [==============================] - 3578s 716ms/step - loss: 0.0724 - acc: 0.6269 - val_loss: 0.0733 - val_acc: 0.6215


In [48]:
print('==>Evaluating Model.')
print(model.evaluate_generator(generator=test_generator, 
    workers=4, use_multiprocessing=True))



==>Evaluating Model.
[0.0723327576557708, 0.6278004453105166]


In [49]:
model.save("deep_cancer_cnn_classifier.h5")
